In [1]:
import pandas as pd
import numpy as np
from utils.io import gdf_merger

In [2]:
coords_cols = ['X', 'Y']

In [3]:
df1 = pd.DataFrame({'ID': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', np.nan, ],
                   'X': [152890, np.nan, 152821, 152815, 152875, 152885, 152810, 152861, 152800],
                   'Y': [122596, np.nan, 122512, 122565, 122575, 122581, 122556, 122523, 122546],
                   'Name': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'i'],
                   'Value': [1, 2, 3, 4, np.nan, np.nan, np.nan, 7, 8]})
df2 = pd.DataFrame({'ID': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', np.nan, np.nan],
                   'X': [152890, 152885, 152821, 152865, 152875, np.nan, 152845, 152861, 152800, 152900],
                   'Y': [122596, 122500, 122556, 122565, 122575, np.nan, 122556, 122523, 122546, 122600],
                   'Name': ['A', 'B', 'D', 'C', 'E', np.nan, 'G', 'h', 'i', 'k'],
                   'Value': [1, 3, 2, 4, np.nan, 5, 6, 7, 8, 9]})
df3 = pd.DataFrame({'ID': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', np.nan, np.nan],
                   'Name': ['A', 'B', 'D', 'C', 'E', 'F', 'G', 'h', 'i', 'k'],
                   'Value2': [1, 3, 2, 4, np.nan, np.nan, 6, 7, 8, 9]})

In [4]:
df1

,ID,X,Y,Name,Value
0,id1,152890.0,122596.0,A,1.0
1,id2,NaN,NaN,B,2.0
2,id3,152821.0,122512.0,C,3.0
3,id4,152815.0,122565.0,D,4.0
4,id5,152875.0,122575.0,E,NaN
5,id6,152885.0,122581.0,F,NaN
6,id7,152810.0,122556.0,G,NaN
7,id8,152861.0,122523.0,H,7.0
8,NaN,152800.0,122546.0,i,8.0


In [5]:
df2

,ID,X,Y,Name,Value
0,id1,152890.0,122596.0,A,1.0
1,id2,152885.0,122500.0,B,3.0
2,id3,152821.0,122556.0,D,2.0
3,id4,152865.0,122565.0,C,4.0
4,id5,152875.0,122575.0,E,NaN
5,id6,NaN,NaN,NaN,5.0
6,id7,152845.0,122556.0,G,6.0
7,id8,152861.0,122523.0,h,7.0
8,NaN,152800.0,122546.0,i,8.0
9,NaN,152900.0,122600.0,k,9.0


In [6]:
data_merger(df1, df3, col='ID')[0]


Columns dropped :['Name_x', 'Name_y']



,ID,X,Y,Value,Value2,Name
0,id1,152890.0,122596.0,1.0,1.0,A
1,id2,NaN,NaN,2.0,3.0,B
2,id3,152821.0,122512.0,3.0,2.0,NaN
3,id4,152815.0,122565.0,4.0,4.0,NaN
4,id5,152875.0,122575.0,NaN,NaN,E
5,id6,152885.0,122581.0,NaN,NaN,F
6,id7,152810.0,122556.0,NaN,6.0,G
7,id8,152861.0,122523.0,7.0,7.0,H
8,NaN,152800.0,122546.0,8.0,8.0,i
9,NaN,152800.0,122546.0,8.0,9.0,NaN


In [7]:
def single_cols(df):
    import re
    
    return list(set([re.sub("_x|_y", "", df.columns.to_list()[x]) for x in range(len(df.columns)) if not
re.compile(r"_x|_y").search(df.columns.to_list()[x])]))

In [8]:
def dble_cols(df):
    import re
    
    return list(set([re.sub("_x|_y", "", df.columns.to_list()[x]) for x in range(len(df.columns)) if
                         re.compile(r"_x|_y").search(df.columns.to_list()[x])]))

In [9]:
mdf = pd.merge(df1,df3, on='ID')
cols = single_cols(mdf) + dble_cols(mdf)

In [10]:
mdf

,ID,X,Y,Name_x,Value,Name_y,Value2
0,id1,152890.0,122596.0,A,1.0,A,1.0
1,id2,NaN,NaN,B,2.0,B,3.0
2,id3,152821.0,122512.0,C,3.0,D,2.0
3,id4,152815.0,122565.0,D,4.0,C,4.0
4,id5,152875.0,122575.0,E,NaN,E,NaN
5,id6,152885.0,122581.0,F,NaN,F,NaN
6,id7,152810.0,122556.0,G,NaN,G,6.0
7,id8,152861.0,122523.0,H,7.0,h,7.0
8,NaN,152800.0,122546.0,i,8.0,i,8.0
9,NaN,152800.0,122546.0,i,8.0,k,9.0


In [11]:
for i in dble_cols(mdf):
    if mdf[f'{i}_x'].dtype !=  mdf[f'{i}_y'].dtype:
        print(f'*** Warning: Type mismatch between {i}_x and {i}_y')
    elif mdf[f'{i}_x'].dtype == 'object':
        mdf[f'conflict_{i}'] = mdf[f'{i}_x'].str.lower()!=mdf[f'{i}_y'].str.lower()
    elif mdf[f'{i}_x'].dtype == 'float64':
        mdf[f'conflict_{i}'] = (mdf[f'{i}_x']!=mdf[f'{i}_y']) & ~(np.isnan(mdf[f'{i}_x']) | np.isnan(mdf[f'{i}_y']))
    else:
        print(f'*** Warning: Unhandled dtype for {i}')

In [14]:
dist_max = 2.

In [15]:
if all(x in dble_cols(mdf) for x in coords_cols):
    mdf['distinct_objects'] = ((mdf['X_x']-mdf['X_y'])**2+(mdf['Y_x']-mdf['Y_y'])**2) > dist_max**2
else:
    mdf['distinct_objects'] = False

In [16]:
mdf

,ID,X,Y,Name_x,Value,Name_y,Value2,conflict_Name,distinct_objects
0,id1,152890.0,122596.0,A,1.0,A,1.0,False,False
1,id2,NaN,NaN,B,2.0,B,3.0,False,False
2,id3,152821.0,122512.0,C,3.0,D,2.0,True,False
3,id4,152815.0,122565.0,D,4.0,C,4.0,True,False
4,id5,152875.0,122575.0,E,NaN,E,NaN,False,False
5,id6,152885.0,122581.0,F,NaN,F,NaN,False,False
6,id7,152810.0,122556.0,G,NaN,G,6.0,False,False
7,id8,152861.0,122523.0,H,7.0,h,7.0,False,False
8,NaN,152800.0,122546.0,i,8.0,i,8.0,False,False
9,NaN,152800.0,122546.0,i,8.0,k,9.0,True,False


## Aggréger les conflits et déclarer un conflit si les objets ne sont pas distincts

In [17]:
mdf['conflict'] = False
for i in dble_cols(mdf):
    mdf['conflict'] = mdf['conflict'] | mdf[f'conflict_{i}']
mdf['conflict'] = mdf['conflict'] &  ~mdf['distinct_objects']
distinct = mdf.query('distinct_objects==True')

In [18]:
mdf

,ID,X,Y,Name_x,Value,Name_y,Value2,conflict_Name,distinct_objects,conflict
0,id1,152890.0,122596.0,A,1.0,A,1.0,False,False,False
1,id2,NaN,NaN,B,2.0,B,3.0,False,False,False
2,id3,152821.0,122512.0,C,3.0,D,2.0,True,False,True
3,id4,152815.0,122565.0,D,4.0,C,4.0,True,False,True
4,id5,152875.0,122575.0,E,NaN,E,NaN,False,False,False
5,id6,152885.0,122581.0,F,NaN,F,NaN,False,False,False
6,id7,152810.0,122556.0,G,NaN,G,6.0,False,False,False
7,id8,152861.0,122523.0,H,7.0,h,7.0,False,False,False
8,NaN,152800.0,122546.0,i,8.0,i,8.0,False,False,False
9,NaN,152800.0,122546.0,i,8.0,k,9.0,True,False,True


In [19]:
for i in dble_cols(mdf):
    distinct.loc[:,f'{i}_x'] = distinct[[f'{i}_x', f'{i}_y', 'distinct_objects']].apply(lambda x: x[1] if x[2]==True else x[0], axis=1)
    if i not in coords_cols:
        mdf[f'{i}_y'] = mdf[[f'{i}_x', f'{i}_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)
mdf_2 = mdf.append(distinct)

In [20]:
distinct

,ID,X,Y,Name_x,Value,Name_y,Value2,conflict_Name,distinct_objects,conflict


In [21]:
mdf_2

,ID,X,Y,Name_x,Value,Name_y,Value2,conflict_Name,distinct_objects,conflict
0,id1,152890.0,122596.0,A,1.0,A,1.0,False,False,False
1,id2,NaN,NaN,B,2.0,B,3.0,False,False,False
2,id3,152821.0,122512.0,C,3.0,D,2.0,True,False,True
3,id4,152815.0,122565.0,D,4.0,C,4.0,True,False,True
4,id5,152875.0,122575.0,E,NaN,E,NaN,False,False,False
5,id6,152885.0,122581.0,F,NaN,F,NaN,False,False,False
6,id7,152810.0,122556.0,G,NaN,G,6.0,False,False,False
7,id8,152861.0,122523.0,H,7.0,h,7.0,False,False,False
8,NaN,152800.0,122546.0,i,8.0,i,8.0,False,False,False
9,NaN,152800.0,122546.0,i,8.0,k,9.0,True,False,True


Dataframe des conflits 

In [22]:
conflicts_to_resolve = mdf_2.query('conflict==True')

In [23]:
conflicts_to_resolve

,ID,X,Y,Name_x,Value,Name_y,Value2,conflict_Name,distinct_objects,conflict
2,id3,152821.0,122512.0,C,3.0,D,2.0,True,False,True
3,id4,152815.0,122565.0,D,4.0,C,4.0,True,False,True
9,NaN,152800.0,122546.0,i,8.0,k,9.0,True,False,True


Dataframe des row sans conflits

In [24]:
mdf_2 = mdf_2.query('conflict==False')

In [25]:
for i in dble_cols(mdf):
    mdf_2[f'{i}'] = mdf_2[[f'{i}_x', f'{i}_y']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

<ipython-input-25-13ae65900246>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf_2[f'{i}'] = mdf_2[[f'{i}_x', f'{i}_y']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)


In [26]:
mdf_2 = mdf_2[cols]

In [27]:
mdf_2.reset_index(drop=True, inplace=True)

In [28]:
mdf_2

,Y,Value,X,ID,Value2,Name
0,122596.0,1.0,152890.0,id1,1.0,A
1,NaN,2.0,NaN,id2,3.0,B
2,122575.0,NaN,152875.0,id5,NaN,E
3,122581.0,NaN,152885.0,id6,NaN,F
4,122556.0,NaN,152810.0,id7,6.0,G
5,122523.0,7.0,152861.0,id8,7.0,H
6,122546.0,8.0,152800.0,NaN,8.0,i


In [29]:
cols

['Y', 'Value', 'X', 'ID', 'Value2', 'Name']

In [30]:
# reste à tester si deux objets aved des id différents ne sont pas en réalité un seul et même objet -> très compliqué à faire sans verification sur le terrain